In [1]:
#загрузка необходимых библиотек
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Парсинг данных по квартирам во Владимирской области

In [2]:
url_st = 'https://vladimir.cian.ru/cat.php?deal_type=sale&engine_version=2&maxtarea=99&offer_type=flat&p='
url_end = '&region=4564&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'


In [3]:
urls = []
ats = []
for i in range(1,55):
    url = requests.get(url_st+str(i)+url_end)
    html = url.text
    bs = BeautifulSoup(html, 'lxml')
    aTags = bs.find_all("a",class_="_93444fe79c--link--39cNw")
    ats.append(aTags)
    

In [4]:
for a in ats:
    for i in a:
        urls.append(i.get('href'))

In [5]:
len(urls)

1509

totsp - общая площадь

kitsp - площадь кухни

livsp  - площадь жилая

floor - этаж

Nfloor - всего этажей

city - город Владимирской области

type - тип дома

year - год постройки

price - цена

room - количество комнат

link - ссылка на сайт циан.ру

overlaps - тип перекрытий

entrance - кол-во подъездов

In [6]:
def write_csv(data):
    with open('Vlad_flat.csv', 'a', encoding = 'Utf-8') as f:
        writer = csv.writer(f)
        writer.writerow((data['totsp'],
                         data['kitsp'],
                         data['livsp'],
                         data['floor'],
                         data['Nfloor'],
                         data['city'],
                         data['htype'],
                         data['year'],
                         data['overlaps'],
                         data['entrance'],
                         data['room'],
                         data['link'],
                         data['price']))


        

        
        
        
def get_flat(link):
    url = requests.get(link)
    html = url.text
    bs = BeautifulSoup(html)

    try:
        div = bs.find_all("div",class_="a10a3f92e9--info-value--18c8R")
        div1 = bs.find_all("div",class_="a10a3f92e9--info-title--2bXM9")
        div_new = []
        for i in range(len(div1)):
            div_new.append(div1[i].text)
        if ('Срок сдачи' in div_new):
            ind = div_new.index('Срок сдачи')
            div.remove(div[ind])
        elif ('Построен' in div_new):
            ind = div_new.index('Построен')
            div.remove(div[ind])
        else:
            pass

        if len(div) == 4:
            totsp = div[0].text.split('\xa0')[0]
            livsp = div[1].text.split('\xa0')[0]
            kitsp = div[2].text.split('\xa0')[0]
            floor = div[3].text.split(' ')[0]
            Nfloor = div[3].text.split(' ')[2]
        if len(div) == 2:
            totsp = div[0].text.split('\xa0')[0]
            floor = div[1].text.split(' ')[0]
            Nfloor = div[1].text.split(' ')[2]
            livsp = ''
            kitsp = ''
        if len(div) == 3:
            totsp = div[0].text.split('\xa0')[0]
            
            if div1[1].text == 'Жилая':
                livsp = div[1].text.split('\xa0')[0]
                kitsp = ''
            else:
                kitsp = div[1].text.split('\xa0')[0]
                livsp = ''
            floor = div[2].text.split(' ')[0]
            Nfloor = div[2].text.split(' ')[2]
    except:
        totsp = ''
        livsp = ''
        kitsp = ''
        floor = ''
        Nfloor = ''
    try:
        span = bs.find_all("span", class_ = "a10a3f92e9--price_value--1iPpd")
        pr = span[0].find("span").get("content").split(' ')[:3]
        price = pr[0]+pr[1]+pr[2]
    except:
        price = 0
    
    div = bs.find_all("div", class_ = "a10a3f92e9--value--38caj")
    div1 = bs.find_all("div", class_ = "a10a3f92e9--name--22FM0")
    div_new = [div1[i].text for i in range(len(div1))]

    try:
        ind = div_new.index('Год постройки')
        year = div[ind].text
    except:
        year = ''
    try:
        ind = div_new.index('Тип дома')
        htype = div[ind].text
    except:
        htype = ''
    try:
        ind = div_new.index('Тип перекрытий')
        overlaps = div[ind].text
    except:
        overlaps = ''
    try:
        ind = div_new.index('Подъезды')
        entrance = div[ind].text
    except:
        entrance = ''

    a = bs.find_all("a", class_ = "a10a3f92e9--link--1t8n1 a10a3f92e9--address-item--1clHr")
    try:
        city = a[1].text
    except:
        city = ''    
   
    h = bs.find_all("h1", class_ = "a10a3f92e9--title--2Widg")
    try:
        room = h[0].text.split('-')[0]
    except:
        room = ''
        
    data = {'totsp': totsp,
            'livsp': livsp,
            'kitsp': kitsp,
            'floor': floor,
            'Nfloor': Nfloor,
            'city': city,
            'year': year,
            'htype': htype,
            'overlaps': overlaps,
            'entrance': entrance,
            'room': room,
            'link': link,
            'price': price}
    write_csv(data)

In [7]:
for url in tqdm(urls):
    get_flat(url)

100%|██████████████████████████████████████████████████████████████████████████████| 1509/1509 [34:00<00:00,  1.28s/it]
